# 6장 19대 대선 결과 분석

## 6-1 Selnium, Beatiful Soup을 이용한 데이터 획득 준비 작업

In [4]:
import pandas as pd
import pandas as np

import platform
import matplotlib.pyplot as plt

%matplotlib inline

path = "C:/Windows/Fonts/Arial.ttf"
from matplotlib import font_manager, rc
if platform.system() == 'Darwin':
    rc('font', family = 'AppleGothic')
elif platform.system() == 'Windows' :
    font_name = font_manager.FontProperties(fname=path).get_name()
else :
    print('Unknown system... sorry~~~')
plt.rcParams['axes.unicode_minus'] = False

## 크롬 드라이버로 접속

In [5]:
from selenium import webdriver
import time

In [7]:
driver = webdriver.Chrome('C:/Users/icaru/Documents/python/Dongwon_2013136129/Ch6/driver/chromedriver.exe')
driver.get("http://info.nec.go.kr/")

WebDriverException: Message: 'chromedriver.exe' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


#### 선거통계 시스템은 frame 사용 -> 개발도구-source에서 확인 가능
#### switch_to_default_content - 에러방지

In [ ]:
driver.switch_to_default_content()
driver.switch_to_frame('main') # selennium 프레임 이동 명량

In [ ]:
driver.find_element_by_xpath("""//*[@id="header"]/ul[1]/li[2]/a""").click()

In [ ]:
driver.find_element_by_xpath("""//*[@id="presubmu"]/li[3]/a""").click()

In [ ]:
driver.find_element_by_xpath("""//*[@id="header"]/div[4]/ul/li[6]/a""").click()

In [ ]:
driver.find_element_by_xpath("""//*[@id="electionType1"]""").click()

### Xpath 안에있는 option이라는 이름의 태그만 모두 가져와서 리스트에 저장

In [ ]:
when_list_raw = driver.find_element_by_xpath("""//*[@id="electionName"]""")
when_list = when_list_raw.find_elements_by_tag_name("option")
when_names_values = [option.text for option in when_list]
when_names_values = when_names_values[1:]
when_names_values

In [ ]:
element = driver.find_element_by_id("electionName")
element.send_keys(when_names_values[0])

In [ ]:
type_list_raw = driver.find_element_by_xpath("""//*[@id="electionCode"]""")
type_list = type_list_raw.find_elements_by_tag_name("option")
type_names_values = [option.text for option in type_list]
type_names_values = type_names_values[1:]
type_names_values

In [ ]:
element = driver.find_element_by_id("electionCode")
element.send_keys(type_names_values[0])

In [ ]:
sido_list_raw = driver.find_element_by_xpath("""//*[@id="cityCode"]""")
sido_list = sido_list_raw.find_elements_by_tag_name("option")
sido_names_values = [option.text for option in sido_list]
sido_names_values = sido_names_values[1:]
sido_names_values

In [ ]:
element = driver.find_element_by_id("cityCode")
element.send_keys(sido_names_values[0])

In [ ]:
sigun_list_raw = driver.find_element_by_xpath("""//*[@id="townCode"]""")
sigun_list = sigun_list_raw.find_elements_by_tag_name("option")
sigun_names_values = [option.text for option in sigun_list]
sigun_names_values =sigun_names_values[1:]
sigun_names_values

In [ ]:
element = driver.find_element_by_id("townCode")
element.send_keys(sigun_names_values[0])

In [ ]:
driver.find_element_by_xpath("""//*[@id="searchBtn"]""").click()

####  필요한 수치들을 수집하기 위해 beautifulSoup 사용

In [ ]:
from bs4 import BeautifulSoup

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

####  td 태그의 alignR 사용

In [ ]:
tmp = soup.find_all('td', 'alignR')
tmp[1:5]

#### 광역시도 리스트와 시도리스트를 얻어 데이터로 만듬

In [ ]:
from tqdm import tqdm_notebook

sido_name_list = []
sigun_name_list = []

for sido_value in tqdm_notebook(sido_names_values):
    element = driver.find_element_by_id("cityCode")
    element.send_keys(sido_value)
    

    
    time.sleep(1)

    sigun_list_raw = driver.find_element_by_xpath("""//*[@id="townCode"]""")
    sigun_list = sigun_list_raw.find_elements_by_tag_name("option")

    sigun_names_values = [option.text for option in sigun_list]
    sigun_names_values = sigun_names_values[1:]
    
    for sigun_value in sigun_names_values:
        sido_name_list.append(sido_value)
        sigun_name_list.append(sigun_value)    

#### 코드 결과를 panda DaraFrame으로 저장

In [ ]:
election_result = pd.DataFrame({'광역시도':sido_name_list, '시군:': sigun_name_list})

election_result.head()

#### 문, 홍, 안 후보의 득표수를 전역변수에 저장

In [ ]:
def get_vote_info(n):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    tmp = soup.find_all('td', 'alignR')
    tmp_values = [float(tmp_val.get_text().replace(',', '')) for tmp_val in tmp[1:5]]
    
    pop[n] = tmp_values[0]
    moon[n] = tmp_values[1]
    hong[n] = tmp_values[2]
    ahn[n] = tmp_values[3]

#### 에러 발생시 실행할 코드, 에러난 항목을 NaN 처리

In [ ]:
import numpy as np

def fail_procedure(n):
    pop[n] = np.nan
    moon[n] = np.nan
    hong[n] = np.nan
    ahn[n] = np.nan

In [ ]:
pop = [np.nan]*len(election_result)
moon = [np.nan]*len(election_result)
hong = [np.nan]*len(election_result)
ahn = [np.nan]*len(election_result)

len(pop), len(moon), len(hong), len(ahn)

In [ ]:
for n in tqdm_notebook(election_result.index):
    try:
        element = driver.find_element_by_id("cityCode")
        element.send_keys(election_result['광역시도'][n])
        
        time.sleep(0.5)
        
        element = driver.find_element_by_id("townCode")
        element.send_keys(election_result['시군'][n])
        
        
        
        driver.find_element_by_xpath("""//*[@id="searchBtn"]""").click()
        
        time.sleep(0.5)
        
        get_vote_info(n) 
    except:
        print("Error")
        fail_procedure(n) 
          

In [ ]:
election_result['pop'] = pop
election_result['moon'] = moon
election_result['ahn'] = ahn
election_result['hong'] = hong

election_result.head()

In [ ]:
re_try_index = election_result[election_result['pop'].isnull()].index

for n in tqdm_notebook(re_try_index):
    try:
        element = driver.find_element_by_id("cityCode")
        element.send_keys(election_result['광역시도'][n])
        
        time.sleep(0.5)
        
        element = driver.find_element_by_id("townCode")
        element.send_keys(election_result['시군'][n])
        
        driver.find_element_by_xpath("""//*[@id="searchBtn"]""").click()
        
        time.sleep(0.5)
        
        get_vote_info(n)
        
    except:
        print('---Error---')
        fail_procedure(n)

In [ ]:
election_result['pop'] = pop
election_result['moon'] = moon
election_result['ahn'] = ahn
election_result['hong'] = hong

election_result.head()

### 결과를 csv 파일로 저장

In [ ]:
election_result.to_csv('./05. election_result.csv', encoding = 'utf-8', sep=',')

### csv 파일 읽어들임

In [ ]:
election_result = pd.read_csv('C:/Users/dwlio/Documents/python/Dongwon_2013136129/Ch6/data/05. election_result.csv', encoding = 'utf-8', 
                              index_col=0)
election_result.head()

### 광역시도 이름을 보기 쉽게 정리

In [ ]:
sido_candi = election_result['광역시도']
sido_candi = [name[:2] if name[:2] in ['서울', '부산', '대구', '광주', '인천', '대전', '울산']
              else '' for name in sido_candi]

#### 이름이 두글자 이면 그대로, 세글자이면 '구'를 뺌

In [ ]:
def cut_char_sigu(name):
    return name if len(name)==2 else name[:-1]

#### 광역시가 아닌데 행정구를 가진 도시들을 정리

In [ ]:
import re

sigun_candi = ['']*len(election_result)

for n in election_result.index:
    each = election_result['시군'][n]
    if each[:2] in ['수원','성남','안양','안산','고양','용인','청주','천안','전주','포항','철원']
        sigun_candi[n] = re.split('시', each)[0] + ' ' + ＼ + cut_char_sigu(re.split('시', each)[1])
    
    else:
        sigun_candi[n] = cut_char_sigu(each)
        
sigun_candi
        

#### 광역시도 이름이 저장된 변수 sido_candi와 위 코드에서 정리한 시구군 이름이 저장된 sigun_candi를 합침
#### ex) 의왕시-> 의왕, 중랑구-> 중랑
#### 세종시는 예외

In [ ]:
ID_candi = [sido_candi[n]+ ' '+sigun_candi[n] for n in range(0, len(sigun_candi))]

ID_candi = [name[1:] if name[0]==' ' else name for name in ID_candi]
ID_candi = [name[:2] if name[:2]=='세종' else name for name in ID_candi]

ID_candi


#### 위에서 만든 ID_candi와 election_result와 합침, 지도를 그리기 위한 과정

In [ ]:
election_result['ID'] = ID_candi
election_result.head(10)

#### 문재인, 홍준표, 안철수의 득표수에서 투표자수를 나누어 득표율 계산

In [ ]:
election_result[['rate_moon', 'rate_hong', 'rate_ahn']] = ＼
            election_result[['moon', 'hong', 'ahn']].div(election_result['pop'], axis=0)
election_result[['rate_moon', 'rate_hong', 'rate_ahn']] *= 100
election_result.head()

#### 문재인의 득표율이 높은 순으로 정렬

In [ ]:
election_result.sort_values(['rate_moon'], ascending=[False]).head(10)

#### 홍준표 득표율이 높은 지역순으로 나열

In [ ]:
election_result.sort_values(['rate_hong'], ascending=[False]).head(10)

#### 안철수 득표율이 높은 지역순으로 나열

In [ ]:
election_result.sort_values(['rate_ahn'], ascending=[False]).head(10)

#### draw_korea.csv 파일을 읽어 전국 지도 그리기

In [ ]:
draw_korea = pd.read_csv('C:/Users/dwlio/Documents/python/Dongwon_2013136129/Ch6/data/05. draw_korea.csv', encoding='utf-8', index_col=0)
draw_korea.head()

#### 지도의 좌표가 맞는지 확인하기 위해 서로 차집합이 공집합인지를 확인

In [ ]:
set(draw_korea['ID'].unique()) - set(election_result['ID'].unique())

In [ ]:
set(election_result['ID'].unique()) - set(draw_korea['ID'].unique())

#### 강원도 고성과 경남 고성을 구분

In [ ]:
election_result[election_result['ID'] == '고성']

In [ ]:
election_result.loc[125, 'ID'] = '고성(강원)'
election_result.loc[233, 'ID'] = '고성(경남)'

election_result[election_result['시군']=='고성군']

#### 지도로 시각화 하기에 이름이 너무 긴 마산회원구는 각각 합포,회원 으로 변경

In [ ]:
election_result[election_result['광역시도'] == '경상남도']

In [ ]:
election_result.loc[228, 'ID'] = '창원 합포'
election_result.loc[229, 'ID'] = '창원 회원'

election_result[election_result['광역시도'] == '경상남도']

#### 2016년 6월에 사라진 소사, 오정, 원미구가 draw_korea에는 존재하지만 election_result에는 없음

In [ ]:
set(draw_korea['ID'].unique()) - set(election_result['ID'].unique())

In [ ]:
set(election_result['ID'].unique()) - set(draw_korea['ID'].unique())

#### 따라서 편의상 부천 인구수를 3으로 나눔

#### 부천 전체데이터 확인

In [ ]:
election_result[election_result['시군'] == '부천시']

#### 데이터 삽입할 index번호 확인, 250 부터 넣어주면 됨

In [ ]:
election_result.tail()

In [ ]:
ahn_tmp = election_result.loc[85, 'ahn']/3
hong_tmp = election_result.loc[85, 'hong']/3
moon_tmp = election_result.loc[85, 'moon']/3
pop_tmp = election_result.loc[85, 'pop']/3

rate_moon_tmp = election_result.loc[85, 'rate_moon']
rate_hong_tmp = election_result.loc[85, 'rate_hong']
rate_ahn_tmp = election_result.loc[85, 'rate_ahn']

election_result.loc[250] = [ahn_tmp, hong_tmp, moon_tmp, pop_tmp,
                           '경기도', '부천시', '부천 소사,
                            rate_moon_tmp, rate_hong_tmp, rate_ahn_tmp]
election_result.loc[251] = [ahn_tmp, hong_tmp, moon_tmp, pop_tmp,
                           '경기도', '부천시', '부천 오정,
                            rate_moon_tmp, rate_hong_tmp, rate_ahn_tmp]
election_result.loc[250] = [ahn_tmp, hong_tmp, moon_tmp, pop_tmp,
                           '경기도', '부천시', '부천 원미,
                            rate_moon_tmp, rate_hong_tmp, rate_ahn_tmp]
                            


In [ ]:
election_result[election_result['시군'] == '부천시']

#### 남아있는 index 85의 부천시는 제거

In [ ]:
election_result.drop([85], inplace=True)
election_result[election_result['시군'] == '부천시']

#### 다시 서로의 차집합을 계산해 차집합이 공집합인 것을 확인

In [ ]:
set(draw_korea['ID'].unique()) - set(election_result['ID'].unique())

In [ ]:
set(election_result['ID'].unique() -draw_korea['ID'].unique())

#### election_result와 draw_korea를 합쳐서 final_elect_data에 저장

#### 문vs홍, 문vs안, 홍vs안 득표율 차이를 계산

In [ ]:
final_elect_data = pd.merge(election_result, draw_korea, how='left', on=['ID'])
final_elect_data.head()

In [ ]:
final_elect_data['moon_vs_hong'] = final_elect_data['rate_moon'] - 
                                    final_elect_data['rate_hong']
final_elect_data['moon_vs_ahn'] = final_elect_data['rate_moon'] - 
                                    final_elect_data['rate_ahn']
final_elect_data['ahn_vs_hong'] = final_elect_data['rate_ahn'] - 
                                    final_elect_data['rate_hong']

#### 경계선 그리기

In [ ]:
BORDER_LINES = [
    [(5,1), (5,2), (7,2), (7,3), (11,3), (11,0)], #인천
    [(5,4), (5,5), (2,5), (2,7), (4,7), (4,9), (7,9),
     (7,7), (9,7), (9,5), (10,5), (10,4), (5,4)], #서울
    [(1,7), (1,8), (3,8), (3,10), (10,10), (10,7),
     (12,7), (12,6), (11,6), (11,5), (12,5), (12,4), 
     (11,4), (11,3)], #경기도
    [(8,10), (8,11), (6,11), (6,12)], #강원도
    [(12,5), (13,5), (13,4), (14,4), (14,5), (15,5),
     (15,4), (16,4), (16,2)], #충청북도
    [(16,4), (17,4), (17,5), (16,5), (16,6), (19,6),
     (19,5), (20,5), (20,4), (21,4), (21,3), (19,3), (19,1)], #전라북도
    [(13,5), (13,6), (16,6)], #대전시
    [(13,5), (14,5)], #세종시
    [(21,2), (21,3), (22,3), (22,4), (24,4), (24,2), (21,2)], #광주
    [(20,5), (21,5), (21,6), (23,6)], #전라남도
    [(10,8), (12,8), (12,9), (14,9), (14,8), (16,8), (16,6)], #충청북도
    [(14,9), (14,11), (14,12), (13,12), (13,13)], #경상북도
    [(15,8), (17,8), (17,10), (16,10), (16,11), (14,11)], #대구
    [(17,9), (18,9), (18,8), (19,8), (19,9), (20,9), (20,10), (21,10)], #부산
    [(16,11), (16,13)], #울산
#    [(9,14), (9,15)],
    [(27,5), (27,6), (25,6)],
]

In [ ]:
def drawkoread(targetData, blockedMap, cmapname):
    gamma = 0.75
    
    whitelabelmin = 20.
    
    datalabel =targetData
    
    tmp_max = max([np.abs(min(blockedMap[targetData])),
                   np.abs(max(blockedMap[targetData]))])
    vmin, vmax = -tmp_max, tmp_max
    
    mapdata = blockedMap.pivot_table(index='y', columns='x', values=targetData)
    masked_mapdata = np.ma.masked_where(np.isnan(mapdata), mapdata)
    
    plt.figure(figsize=(9, 11))
    
    plt.pcolor(masked_mapdata, vmin=vmin, vmax=vmax, cmap=cmapname,
              edgecolor='#aaaaaa', linewidth=0.5)

  # 지역 이름 표시
for idx, row in draw_korea.iterrows():
    
    # 광역시는 구 이름이 겹치는 경우가 많아서 시단위 이름도 같이 표시한다. 
    #(중구, 서구)
    if len(row['ID'].split())== 2:
        dispname = '{}\n{}'.format(row['ID'].split()[0], row['ID'].split()[1])
    elif row['ID'][:2]=='고성':
        dispname = '고성'
    else:
        dispname = row['ID']

    # 서대문구, 서귀포시 같이 이름이 3자 이상인 경우에 작은 글자로 표시한다.
    if len(dispname.splitlines()[-1]) >= 3:
        fontsize, linespacing = 10.0, 1.1
    else:
        fontsize, linespacing = 11, 1

    annocolor = 'white' if np.abs(row[targetData]) > whitelabelmin else 'black'
    plt.annotate(dispname, (row['x']+0.5, row['y']+0.5), weight='bold',
                 fontsize=fontsize, ha='center', va='center', color=annocolor,
                 linespacing=linespacing)
    
# 시도 경계 그린다.
for path in BORDER_LINES:
    ys, xs = zip(*path)
    plt.plot(xs, ys, c='black', lw=2)

plt.gca().invert_yaxis()
#plt.gca().set_aspect(1)
#y축이 엑셀에서 0번이 시작하는 것과 matplotlib가 0이라고 인식하는 좌표가 서로 반대이기 때문

plt.axis('off')

cb = plt.colorbar(shrink=.1, aspect=10)
cb.set_label(datalabel)

plt.tight_layout()
plt.show()

#### 문재인 vs 홍준표

In [ ]:
drawKorea('monn_vs_hong', final_elect_data, 'RdBu')

#### 문재인 vs 안철수

In [ ]:
drawKorea('monn_vs_ahn', final_elect_data, 'RdBu')

#### 안철수 vs 홍준표

In [ ]:
drawKorea('ahn_vs_ahn', final_elect_data, 'RdBu')

In [ ]:
import folium
import json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
pop_folium = final_elect_data.set_index('ID')

del pop_folium['광역시도']
del pop_folium['시군']

pop_folium.head()

In [ ]:
geo_path = 'C:/Users/dwlio/Documents/python/Dongwon_2013136129/Ch6/data/05. skorea_muncipalities_geo_simple.json'
geo_data = json.load(open(geo_path, encoding='utf-8'))

map = folium.Map(location =[36.2002, 127.054], zoom_start=6)
map.choropleth(geo_str = geo_str,
              data = pop_folium['moon_vs_hong'],
              columns = [pop_folium.index, pop_folium['monn_vs_hong']],
              fill_collor = 'PuBu', #PuRd, YlGnBu
              key_on = 'feature.id')

map